# MNIST

Goal: train classifier with 4% error in <1 minute on a laptop

## Dataset

In [7]:
from sklearn import datasets
from sklearn import cross_validation
import numpy as np

## Loss functions

In [ ]:
# Multiclass classification
def one_hot(y, k):
    m = len(y)
    return sp.coo_matrix((np.ones(m), (np.arange(m), y)), shape=(m, k)).todense()

def softmax_loss(Theta, X, y):
    k = Theta.size[1]
    return (cp.sum_entries(cp.log_sum_exp(X*Theta, axis=1)) -
            cp.sum_entries(cp.mul_elemwise(one_hot(y, k), X*Theta)))

def multiclass_hinge_loss(Theta, X, y):
    n, k = Theta.size
    ones = np.ones((k,k))
    Y = one_hot(y, k)
    Z = X*Theta
    ZY = cp.mul_elemwise(Y, Z)*ones
    return cp.sum_entries(cp.max_entries(Z - ZY + (1-Y), axis=1))

## Random features

In [10]:
mnist = datasets.fetch_mldata("MNIST original")
Xt, Xs, yt, ys = cross_validation.train_test_split(
    mnist["data"], mnist["target"], train_size=60000, random_state=0)

In [18]:
np.round(lr.predict(Xs))

array([ 2.,  8.,  2., ...,  5.,  3.,  7.])

In [22]:
from sklearn import linear_model

def err(x, y):
    return 1 - np.sum(x == y) / float(len(x))

lr = linear_model.RidgeCV()
lr.fit(Xt,yt)
print "Error:", err(ys, np.round(lr.predict(Xs)))

Error: 0.7724
